In [ ]:
import nltk
import os
import string
import re
import numpy as np
from numpy.linalg import norm
import random
from nltk.corpus import stopwords

In [2]:
class SVM():
    
    def __init__(self, trainingData, testData, testLabels):
        self.trainingData = trainingData
        self.testData = testData
        self.testLabels = testLabels
        self.h = []
        self.testh = []

    def setupHVectors(self, trainingData, testData):
        dataTuple = (trainingData, testData)
        symbolSet = []
        d = 100
        H = []
        h = []
        for data in dataTuple:
            symbolLst = self.makeSymbolList(data)
            trigramLst = self.makeTriGramList(symbolLst)
            (N, S) = self.makeNAndS(trigramLst)
            eachSymbolSet = set(symbolLst)
            newSymbols = [x for x in eachSymbolSet if x not in symbolSet]
            for x in self.createHDVectorForSeveralCharacters(d, newSymbols):
                H.append(x)
            for x in newSymbols:
                symbolSet.append(x)
            eachh = self.createHVector(symbolSet, H, N, S)
            h.append(h)


        h = self.createHVector()
        return h
    def predict(self, h, testh, labels):
        num_vectors = testh.shape[0]
        predictedLabels = np.zeros(num_vectors, dtype=labels.dtype)
        for i in range(num_vectors):
            A = h
            B = testh[i]
            cosine = np.dot(A,B) / norm(A, axis=1)*norm(B)
            maxIndex = np.argmax(cosine)
            predictedLabels[i] = labels[maxIndex]
        return predictedLabels

    def makeSymbolList(self, lst):
        languageSymbolLst = []
        for y in lst:
            for z in list(y):
                languageSymbolLst.append(z)
        return np.asarray(languageSymbolLst)

    def makeTriGramList(self, symbolLst):
        languageTriGramLst = []
        for i in range(1,len(symbolLst)-1):
            triplet = (symbolLst[i-1], symbolLst[i], symbolLst[i+1])
            languageTriGramLst.append(triplet)
        return np.asarray(languageTriGramLst)

    def makeNAndS(self, triGramLst):
        unique = np.unique(triGramLst, axis=0, return_counts=True)
        return (unique[0], unique[1])

    def createHDVector(self, d):
        newHDVector = np.zeros(d, dtype=int)
        for i in range(d):
            rnd = random.randint(0,1)
            if rnd == 0:
                rnd = -1
            newHDVector[i] = rnd
        return np.asarray(newHDVector)

    def createHDVectorForSeveralCharacters(self, d, symbolSet):
        H = np.zeros((len(symbolSet), d))
        for i in range(len(symbolSet)):
                H[i] = self.createHDVector(d)
        return np.asarray(H)

    def bindVectors(self, H, N, symbolSet):
        mN = []
        seed = 42
        for k in range(len(N)): # EACH TRIPLET
            oldTriplet = N[k]
            newTriplet = []
            for x in oldTriplet: # EACH PLACE IN TRIPLET
                for y in range(len(symbolSet)): # FIND SYMBOL FOR EACH INDEX IN TRIPLET
                    if x == list(symbolSet)[y]:
                        newH = np.roll(H[y], len(newTriplet)+1)
                        newTriplet.append(newH)
                        break                   
            mN.append(np.multiply(newTriplet[0], newTriplet[1], newTriplet[2]))
        return np.asarray(mN)
    
    def createHVector(self, symbolSet, H, N, S):
        mN = self.bindVectors(H, N, symbolSet)
        h = np.empty(mN.shape[1])
        for i in range(mN.shape[0]):
            addmNS = mN[i] * S[i]
            h = np.add(h, addmNS)
        return np.asarray(h)  